Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from yeast8model import Yeast8Model

Initialise model

In [ ]:
glc_exch_rate = 16.89
wt_ec = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
wt_ec.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt_ec.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

Ablate and store fluxes in each round

In [ ]:
wt_ec.ablation_result = wt_ec.ablate()

ablation_fluxes = wt_ec.ablation_fluxes
ablation_fluxes_diff = ablation_fluxes.copy()
ablation_fluxes_diff.pop("original")
for biomass_component, fluxes in ablation_fluxes_diff.items():
    ablation_fluxes_diff[biomass_component] = (
        ablation_fluxes[biomass_component] - ablation_fluxes["original"]
    )
    print(f"{biomass_component}")
    print(f"min {1e5 * ablation_fluxes_diff[biomass_component].min()} * 1e-5")
    print(f"max {1e5 * ablation_fluxes_diff[biomass_component].max()} * 1e-5")

(Other work -- rename sections later)

In [ ]:
# Choose component
biomass_component = 'carbohydrate'
s = ablation_fluxes_diff[biomass_component].copy()

# Get top (False) or bottom (True) N
n = 50
s = s.sort_values(ascending=False)[:n]

# Extract protein IDs
rxn_idx_list = s.index.to_list()
enz_metabolite_ids = [rxn_idx.replace("draw_", "") + "[c]" for rxn_idx in rxn_idx_list]

# Print participating reactions
for enz_metabolite_id in enz_metabolite_ids:
    print(enz_metabolite_id)
    participating_rxns = list(wt_ec.model.metabolites.get_by_id(enz_metabolite_id)._reaction)
    for participating_rxn in participating_rxns:
        print(participating_rxn.name)
    print('\n')